In [6]:
# some standard packages
import numpy as np
import pickle
import matplotlib.pyplot as plt

# modelling packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model

Utilize the cell below only if using all of the Data Loader file instead of using the Keras Image Generator.

In [ ]:
# Open the data files
# X = pickle.load(open('X.pickle', 'rb'))
# y = pickle.load(open('y.pickle', 'rb'))

In [7]:
# Use the Keras ImageDataGenerator for memory efficiency and preprocessing ease
# This process replaces the method of obtaining our data via DataLoader.ipynb
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2,
)

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
# train_img_dir = 'Data/train/'
# test_img_dir = 'Data/test/'

In [10]:
batch_size = 128

In [11]:
# Open the data files
train = pickle.load(open('train_labels.pickle', 'rb'))
test = pickle.load(open('test_labels.pickle', 'rb'))

In [12]:
train.head()

,Filename,Label,Species,path
0,20160928-140314-0.jpg,0,Chinee apple,Data/train/class_0/20160928-140314-0.jpg
1,20160928-140337-0.jpg,0,Chinee apple,Data/train/class_0/20160928-140337-0.jpg
2,20160928-140731-0.jpg,0,Chinee apple,Data/train/class_0/20160928-140731-0.jpg
3,20160928-140747-0.jpg,0,Chinee apple,Data/train/class_0/20160928-140747-0.jpg
4,20160928-141107-0.jpg,0,Chinee apple,Data/train/class_0/20160928-141107-0.jpg


In [13]:
train['Label'].value_counts()

8    7287
0     918
4     852
6     851
1     830
7     819
5     818
3     817
2     815
Name: Label, dtype: int64

In [14]:
train.shape

(14007, 4)

In [21]:
train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='path',
    y_col='Label',
    target_size=(28, 28),
    batch_size=batch_size,
    class_mode='input',
    subset='validation')

Found 2801 validated image filenames.


In [22]:
validation_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='path',
    y_col='Label',
    target_size=(28, 28),
    batch_size=batch_size,
    class_mode='input',
    subset='validation')

Found 2801 validated image filenames.


In [17]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y) 

In [18]:
# # Building the Model
model = Sequential()

# # 3 convolutional layers
model.add(Conv2D(32, (3,3), input_shape = (28,28,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# # 2 hidden layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Activation('relu'))

# # The output layer with 9 neurons for 9 classes
model.add(Dense(9))
model.add(Activation('softmax'))

# # Compiling the model using some basic parameters
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
train_generator.filenames

['Data/train/class_0/20160928-140314-0.jpg',
 'Data/train/class_0/20160928-140337-0.jpg',
 'Data/train/class_0/20160928-140731-0.jpg',
 'Data/train/class_0/20160928-140747-0.jpg',
 'Data/train/class_0/20160928-141107-0.jpg',
 'Data/train/class_0/20160928-141135-0.jpg',
 'Data/train/class_0/20160928-141355-0.jpg',
 'Data/train/class_0/20160928-141421-0.jpg',
 'Data/train/class_0/20160928-141437-0.jpg',
 'Data/train/class_0/20160928-142056-0.jpg',
 'Data/train/class_0/20160928-142110-0.jpg',
 'Data/train/class_0/20161207-110753-0.jpg',
 'Data/train/class_0/20161207-110822-0.jpg',
 'Data/train/class_0/20161207-110850-0.jpg',
 'Data/train/class_0/20161207-110901-0.jpg',
 'Data/train/class_0/20161207-110913-0.jpg',
 'Data/train/class_0/20161207-111058-0.jpg',
 'Data/train/class_0/20161207-111112-0.jpg',
 'Data/train/class_0/20161207-111122-0.jpg',
 'Data/train/class_0/20161207-111143-0.jpg',
 'Data/train/class_0/20161207-111159-0.jpg',
 'Data/train/class_0/20161207-111232-0.jpg',
 'Data/tra

In [23]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=(2801) // batch_size,
                    epochs=500, 
                    validation_data=validation_generator,
                    validation_steps=len(2807) // batch_size,
                    callbacks=[
                        EarlyStopping(patience=3, restore_best_weights=True),
                        ReduceLROnPlateau(patience=2)],
                    verbose=1)

IndentationError: unexpected indent (<ipython-input-23-d6e9c0378c18>, line 4)